In [23]:
import pandas as pd
import numpy as np


In [24]:
# read from excel

df = pd.read_excel('卡牌数据&数学模型.xlsx', sheet_name='动物牌')

In [25]:
df = df[df['编号'].notna()] # 过滤无效信息

# float to 3-char string, such as 1.0 -> '001'
df['编号'] = df['编号'].apply(lambda x: str(int(x)).zfill(3))

df

,编号,英文名,中文名,拉丁文,拉丁文idea,危险等级,体量,岩石,水域,金钱成本,条件,图标,特殊饲养区,Unnamed: 13,效果,Unnamed: 15,声望,魅力,保护点数,Unnamed: 19
1,001,Bowtruckle,护树罗锅,habitatio membri arboris,dwelling limb of tree,XX,1.0,NaN,1,14,research,"Europe, Herbivore",-,-,Camauflage,Ignore 1 condition when you play another anima...,0.0,2.0,1.0,NaN
3,002,Niffler,嗅嗅,thesaurus amans,treasure lover,XXX,2.0,NaN,NaN,17,-,"Europe, Herbivore",NaN,NaN,Gold Digging 寻宝,Gain the same amount of money as the square of...,0.0,4.0,0.0,NaN
5,003,Centaur,马人,centauri,Centaur,XXXX,4.0,NaN,NaN,27,2 universities,"Europe, herbivore, primate",-,-,Divination,"Take a look at the top 7 cards from the deck, ...",4.0,7.0,0.0,NaN
6,004,Kappa,卡巴,Iaponica aqua daemonium,water demon,XXXX,3.0,NaN,1,16,2 research,"Asia, primate",NaN,NaN,Appeasement,"Discard a card from display, if it has a water...",0.0,2.0,2.0,NaN
7,005,Erupment,毒角犀,erumpens,erupment,XXXX,4.0,NaN,NaN,26,partner zoo,"Africa, Africa, herbivore",NaN,NaN,Explosion,You may flip the kiosks/pavilions in your zoo....,NaN,9.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,121,NaN,兔蝠,vespertilio-lepus ales,bat-winged rabbit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,从牌库顶补充手牌至手牌上限。,NaN,NaN,NaN,NaN
150,122,NaN,狼鲨,lupus turpis,wolf shark,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,丢弃展示区所有由你选择的1种动物牌，获得对应数量的魅力。,NaN,NaN,NaN,NaN
151,123,NaN,羱狒,simia-cauda ibex,monkey-tail ibex,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,将休息标记推进3格，获得三元。,NaN,NaN,NaN,NaN
152,124,NaN,蟹猩,cancer-unguis nire,crab-claw baboon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,拆除相邻动物园内的一个1/2格建筑，获得它的花费。,NaN,NaN,NaN,NaN


In [26]:
# 标志转化函数
mapping = {
    'research': 'OtherTag.Science',
    'university': 'OtherTag.University',
    'rock': 'OtherTag.Rock',
    'water': 'OtherTag.Water',
    'asia': 'ContinentTag.Asia',
    'europe': 'ContinentTag.Europe',
    'africa': 'ContinentTag.Africa',
    'ameri': 'ContinentTag.Americas',
    'australia': 'ContinentTag.Australia',
    'dragon': 'AnimalTag.Dragon',
    'bird': 'AnimalTag.Bird',
    'predator': 'AnimalTag.Predator',
    'reptile': 'AnimalTag.Reptile',
    'herbivore': 'AnimalTag.Herbivore',
    'primate': 'AnimalTag.Primate',
    'petting': 'AnimalTag.Pet',
    'bear': 'AnimalTag.Bear',
    'sea': 'AnimalTag.SeaAnimal',
    '嵌合': 'AnimalTag.Chimera',

}

def handle_nan_and_dash(func, return_value = ''):
    def wrapper(s):
        if pd.isna(s) or s == '-':
            return return_value
        return func(s)
    return wrapper

# 定义原始的 string_tag_to_ts_enum 函数
def string_tag_to_ts_enum(s) -> list:
    enum_tags = []
    for k, v in mapping.items():
        if s.lower().find(k.lower()) != -1:
            enum_tags.append(v)
    return enum_tags

# 使用装饰器处理 NaN 或者 '-' 值
string_tag_to_ts_enum = handle_nan_and_dash(string_tag_to_ts_enum, [])

# 将处理后的结果赋值给新列 'tags'
df = df.assign(tags=df['图标'].apply(string_tag_to_ts_enum))
df = df.assign(requirements=df['条件'].apply(string_tag_to_ts_enum))
df.drop_duplicates(subset='编号', keep='first', inplace=True)
df.tail(20)

,编号,英文名,中文名,拉丁文,拉丁文idea,危险等级,体量,岩石,水域,金钱成本,...,特殊饲养区,Unnamed: 13,效果,Unnamed: 15,声望,魅力,保护点数,Unnamed: 19,tags,requirements
133,106,Dobby,多比,libera dryadalis,a free elf,NaN,2.0,NaN,NaN,NaN,...,NaN,NaN,工钱,将你的饲养员替换为家养小精灵。,1.0,NaN,1.0,NaN,[],[]
135,107,Kreacher,克利切,inferiorem dryadalis,lower elf,NaN,2.0,NaN,NaN,NaN,...,NaN,NaN,工钱,将你的饲养员替换为家养小精灵。,NaN,NaN,NaN,NaN,[],[]
136,108,Winky,闪闪,puella dryadalis,girl elf,NaN,2.0,NaN,NaN,NaN,...,NaN,NaN,工钱,将你的饲养员替换为家养小精灵。,NaN,NaN,NaN,NaN,[],[]
137,109,Cockatrice,鸡蛇,Cockatrice,Cockatrice,NaN,1.0,NaN,NaN,0.204545,...,NaN,NaN,Gazing 瞪视,All animals except Phoenix played before this ...,NaN,NaN,NaN,NaN,[],[]
138,110,Boggart,博格特,quid,anything,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,百变,复制场上任意1张1动的所有图标和效果。,NaN,NaN,NaN,NaN,[],[]
139,111,NaN,鹤蜥,lacerta-gruis,part-lizard part-crane,NaN,NaN,NaN,NaN,8.8,...,NaN,NaN,NaN,丢弃展示区的1张牌，免费建造1个休憩。,NaN,NaN,NaN,NaN,[],[]
140,112,NaN,狮鹫,gryphem,griffin,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,展示弃牌堆或牌库顶的至多3张牌，将1张动物牌加入你的手牌，丢弃其余的牌。,NaN,NaN,NaN,NaN,[],[]
141,113,NaN,鹦猿,parrot-simia,parrot-monkey,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,抢夺魅力点数最高动物园内的1个黑亭/红亭。,NaN,NaN,NaN,NaN,[],[]
142,114,NaN,鹿鸮,noctua alces,owl-moose,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,展示牌库顶的X张牌，将所有赞助加入你的手牌，X为你动物园内的鸟标+草标。,NaN,NaN,NaN,NaN,[],[]
143,115,NaN,鳞鹳,piscis eu ciconia,fish-scale stork,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,在任意1格水域上免费建造黑亭/红亭。,NaN,NaN,NaN,NaN,[],[]


In [27]:
def nan_to_zero(str):
    if pd.isna(str):
        return 0
    return str
# 定义生成文本内容的函数
def generate_text(row):
    tags = '['+', '.join(row['tags']) + ']'
    requirements = '[' + ', '.join(row['requirements']) + ']'
    reputation = nan_to_zero(row['声望'])
    appeal = nan_to_zero(row['魅力'])
    conservationPoint = nan_to_zero(row['保护点数'])
    size = nan_to_zero(row['体量'])
    price = nan_to_zero(row['金钱成本'])
    new_string = f"""
  {{
    id: '{row['编号']}',
    name: "{row['英文名']}",
    cnName: "{row['中文名']}",
    latinName: "{row['拉丁文']}",
    image: '{row['编号']}{row['中文名']}.png',
    size: {size},
    price: {price},
    rock: 0,
    water: 0,
    requirements: {requirements},
    tags: {tags},
    abilities: [],
    description: '',
    reefDwellerEffect: [],
    soloEffect: [],
    reputation: {reputation},
    appeal: {appeal},
    conservationPoint: {conservationPoint},
    source: CardSource.BASE,
  }},
"""
    return new_string

with open('animals.txt', 'w') as f:
    for index, row in df.iterrows():
        new_string = generate_text(row)
        f.write(new_string)


In [28]:
# 生成 CSS 图片

# 定义生成文本内容的函数
def generate_text(row):
    id = row['编号']
    cnName = row['中文名']
    new_string = f"""
    .zoo-card.animal-card[data-id='{id}'] .ark-card-wrapper {{
  background-image: url(/img/animals/{id}{cnName}.png);

}}
"""
    return new_string

with open('animals_css.txt', 'w') as f:
    for index, row in df.iterrows():
        new_string = generate_text(row)
        f.write(new_string)

